In [ ]:
# Makes a table in which each metagenome is converted to a table of [function, number]
# the lines of the table are written to a "Clean" file

import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')

search_genes = 'ab' # ab: antibiotic resistance; tx: toxins; mt: heavy metals

infiles = [ '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/1.1.Vir.mgm4876242.3.csv', 
            '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/2.3.Vir.mgm4876251.3.csv',
            '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/3.2.Vir.mgm4876251.3.csv',
            '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/4.1.Vir.mgm4876246.3.csv',
            '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/5.3.Vir.mgm4876239.3.csv',
            '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/6.2.Vir.mgm4876231.3.csv'
           ]
labels =  ['S1', 'S2', 'S3', 'S4', 'S5', 'S6'] 
rast_totals = [196401, 102523, 117807, 63138,100590, 101290]

if search_genes == 'ab':
  title = 'Antibiotic Resistance Genes'
  nots = 'empty'
  targets = [ 
           ['Vancomycin'], ['Acriflavine'], ['Fosfomycin'],
           ['Streptomycin'], ['Azithromycin'], ['Lincomycin'], 
           ['Trimethoprim'], ['Rifam'] , ['Aminoglycoside'],
           ['Metronidazole'], ['Nitroimidazole'],  ['Quinolone'], ['Macrolide'], ['quninolon'],            
           ['Polymyxin'], ['Topoisomerase inhibitor'], ['Beta-lactamase'], ['mycin'], ['Multidrug'] ]  #

elif search_genes == 'mt':
  targets = [ ['Cobalt'], ['Mercury'] , ['Zinc'], ['Arsenic'], ['Cadmium'], ['Copper'], 
             ['Chromate'], ['Lead'], ['Co/Zn/Cd'] ]
  title = 'Heavy Metal Resistance Genes'
  nots = 'precorrin'

elif search_genes == 'tx': 
  targets = [ ['Toxin'], ['Hemolysin'], ['Leukocidin'], ['Collagenase'], ['Hyaluronate lyase'], 
             ['Fibronectin'], ['Type iii'],['Multiple'] ]
  title = 'Toxin Genes'
  nots = 'empty'

# for reading files and converting each taxon to a list of [taxon, number]
clean_lines =[]
for x in range(len(infiles)):
  with open(infiles[x], 'r') as f:
    data1 = f.read()
    data2 = data1.lower().replace('co/zn/cd', 'cobalt, zinc, cadmium').\
    replace('gyrase', 'topoisomerase inhibitor').replace('erythromycin', 'macrolide').\
    replace('acriflavin', 'acriflavine').replace('macrolide', 'macrolide').\
    replace('polymyxin', 'aminoglycoside').replace('topoisomerase', 'topoisomerase inhibitor').\
    replace('mercuric', 'mercury')
    #print(repr(data[-100:]))
    lines = data2.split('\n')
    long_lines =[]
    for y in range(len(lines)):
      if len(str(lines[y])) >120: 
        long_lines.append(lines[y])    
    clean_lines.append(long_lines)
    print('Number of genes at Site', x+1, '=', len(long_lines))
  f.close()

multiples =[]
for x in range(len(targets)-1):
  multiples.append(targets[x][0])
print('\nList for finding multiples:', multiples)
print()

# make table with each row being  a list with one target and 'numer of files' 0's
for a in range(len(targets)):
  for b in range(len(infiles)):
    targets[a].append(0)

funcset = set() 
dup_hits, mon_hits, total_hits = 0, 0, 0 
for a in range(len(clean_lines)):
  for b in range(len(clean_lines[a])):
    for c in range(len(targets)):
      annotation = clean_lines[a][b]  
      if targets[c][0].lower() in annotation and annotation not in funcset and\
                 nots not in annotation: 
        total_hits +=1
        funcset.add(annotation)
        duplicate = False
        for d in range(c+1, len(multiples)):
          if multiples[d].lower() in annotation:
            duplicate = True 
            break                      
        if duplicate == False: 
          targets[c][a+1] +=1 / len(clean_lines[a])
          mon_hits +=1
        elif duplicate == True:  
          targets[len(targets)-1][a+1] += 1 / len(clean_lines[a])
          dup_hits +=1  

for x in range(len(targets)):
  targets[x].append(sum(targets[x][1:]))
#for x in targets: print(x)
numbars = len(targets[0])-1
targets.sort(key=lambda x: x[numbars] )
targets.reverse()
#for x in targets: print(x)
for x in range(len(targets)):
  targets[x].pop()

numfunc =0
plotfunc =[]
for a in range(len(targets)):
  positives =0
  for b in range(1,len(targets[a])):
    if targets[a][b] >0: 
      positives +=1
  if positives >= 0.6 *len(labels) : 
    numfunc +=1
    plotfunc.append(targets[a])
print()
print('\ntotal_hits =', total_hits, ', dup_hits =', dup_hits, ', mon_hits =', mon_hits)

numsites =len(labels)
barlines =[]
colors = ['r', 'c', '#ffde00', 'g', 'b', '#7cd700', 'm','#400000', '#8f6d5f', '#ff0087', '#33FF90'] 

for x in range(len(plotfunc)):
  for y in range(1, len(plotfunc[x])):
    plotfunc[x][y] = float(plotfunc[x][y])
plotname =[]
for x in range(numfunc):
  plotname.append(plotfunc[x][0])

for x in range(numfunc):
  barlines.append(plotfunc[x][1:])
  print(plotfunc[x])
width = 0.5       # the width of the bars: can also be len(x) sequence
fig, ax = plt.subplots(dpi=300)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

bottomsum = [] 
for x in range(len(labels)):
  bottomsum.append(0)

for x in range(len(plotname)):
  if x == 0:
    ax.bar(labels, barlines[x], width, color= colors[x], bottom= bottomsum, label=plotname[x]) 
  else:
    ax.bar(labels, barlines[x], width, color= colors[x % len(colors)], 
           bottom=[sum(z) for z in zip(bottomsum, barlines[x-1]) ], label=plotname[x]) 
    bottomsum = [sum(z) for z in zip(bottomsum, barlines[x-1]) ] 

ax.set_ylabel('Frequencies', fontsize=14)
ax.set_xlabel('Site', fontsize=14)
ax.set_title(title, fontsize = 16)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
plt.show()
print('\nDone.')